In [1]:
import os
os.chdir('/home/xyz/data/haoyan/bandits')

import bandits as bd
import numpy as np
import pandas as pd

/home/xyz/miniconda3/envs/bandit/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [61]:
opt = 20
func_name = 'mix'

In [67]:
from ast import literal_eval as make_tuple


l = []
names = ['random_state', 'num_exp', 'reward', 'time']
with open(f'/home/xyz/data/haoyan/bandits/logs/grid_search_results_{func_name}.txt', 'r') as f:
    x = f.readline()
    while x:
        idx = 0
        tmp = make_tuple(x)
        tmp_dict = {
            "random_state": tmp[0],
            'num_exp': tmp[1],
            'reward': tmp[3]
        }
        tmp_dict.update(tmp[2])
        l.append(tmp_dict)
        x = f.readline()

In [71]:
df = pd.DataFrame(l).reset_index(drop=True)
idx = df.groupby('random_state')['reward'].idxmax()
df.loc[idx]

,random_state,num_exp,reward,u,x,y
174,0,174,19.997670,2,0,-0.048275
993,128,493,19.999358,2,0,-0.025328
1127,256,127,19.953615,2,0,-0.215373
1663,384,163,19.984439,2,0,-0.124742
2051,512,51,19.993705,2,0,-0.079343


In [72]:
import plotly.graph_objs as go

fig = go.Figure()
moving_avg_list = []
for key, gp in df.groupby('random_state'):
    moving_avg_list.append(abs(gp.sort_values('num_exp')['reward'] - opt))

for x in moving_avg_list[-2:-1]:
    fig.add_trace(go.Scatter(x=list(range(len(x))), y=x, mode='markers', marker={'size': 3}, name='Loss'))
    fig.add_trace(go.Scatter(x=list(range(len(x))), y=x.rolling(50).mean(), mode='lines', name='Rolling 50 mean'))
    break

fig.update_layout(
    title={
        "text": "One Experiment Random Search Convergence Result",
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Number of Iteration",
    yaxis_title="L1 Loss Between Searched and Func Opt",
)

# fig.show()

In [74]:
fig.write_image("/home/xyz/data/haoyan/bandits/bandits/image/mix/random_search.png")

In [75]:
from ast import literal_eval as make_tuple


l = []
last_random_state = -99999
names = ['random_state', 'num_exp', 'reward']
with open(f'/home/xyz/data/haoyan/bandits/logs/rl_hybrid_results_{func_name}.txt', 'r') as f:
    x = f.readline()
    while x:
        tmp = make_tuple(x)
        if last_random_state != tmp[0]:
            idx = 0
            last_random_state = tmp[0]
        for element in tmp[3]:
            tmp_dict = {
                'random_state': tmp[0],
                'nth_trial': tmp[1],
                'num_exp': idx,
                'u': tmp[2][0][0],
                'v': tmp[2][0][1],
            }
            tmp_dict.update(element['params'])
            tmp_dict['bayesian_opt'] = element['target']
            tmp_dict['reward'] = tmp[4]
            l.append(tmp_dict)
            idx += 1
        x = f.readline()

In [76]:
df = pd.DataFrame(l).reset_index(drop=True)
idx = df.groupby('random_state')['bayesian_opt'].idxmax()
df.loc[idx]

,random_state,nth_trial,num_exp,u,v,y,bayesian_opt,reward
275,0,91,275,2,0,-0.003537,19.999987,19.999987
759,128,121,363,2,0,0.000163,20.000000,20.000000
1011,256,42,126,2,0,-0.003366,19.999989,19.999989
1388,384,28,86,2,0,0.000082,20.000000,20.000000
2015,512,99,299,2,0,-0.000163,20.000000,20.000000


In [77]:
df.groupby('random_state')['reward'].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

,count,mean,std,min,25%,50%,75%,90%,95%,99%,max
random_state,,,,,,,,,,,
0,396.0,9.521185,11.237957,-26.354195,4.788533,11.000000,19.000000,19.999987,19.999987,19.999987,19.999987
128,489.0,11.173924,10.287299,-24.250106,6.575502,16.000000,19.000000,20.000000,20.000000,20.000000,20.000000
256,417.0,10.242227,9.240141,-21.562783,5.072026,9.999274,18.999999,19.999989,19.999989,19.999989,19.999989
384,414.0,9.046099,10.711350,-21.485541,2.375724,10.891499,18.999999,19.000000,20.000000,20.000000,20.000000
512,402.0,8.651292,8.342098,-18.018572,4.341453,9.139851,16.000000,18.995215,18.999999,20.000000,20.000000


In [78]:
df['reward-opt'] = (df['bayesian_opt'] - opt).abs()
df['abs(reward-opt)<0.05'] = df['reward-opt'] < 0.05
df['abs(reward-opt)<0.1'] = df['reward-opt'] < 0.1
df['abs(reward-opt)<0.2'] = df['reward-opt'] < 0.2
tmp = df.groupby('random_state')[['abs(reward-opt)<0.05', 'abs(reward-opt)<0.1', 'abs(reward-opt)<0.2']].sum() / df.groupby('random_state').size().values.reshape((-1,1)) * 100
tmp.index.name = 'RL Hybrid'
tmp

,abs(reward-opt)<0.05,abs(reward-opt)<0.1,abs(reward-opt)<0.2
RL Hybrid,,,
0,22.727273,22.979798,22.979798
128,17.995910,18.200409,18.404908
256,20.143885,20.623501,20.863309
384,6.280193,6.521739,6.521739
512,2.487562,2.487562,2.736318


In [79]:
import plotly.graph_objs as go

fig = go.Figure()
moving_avg_list = []
for key, gp in df.groupby('random_state'):
    moving_avg_list.append(abs(gp.sort_values('num_exp')['bayesian_opt'] - opt))

for x in moving_avg_list[0:1]:
    fig.add_trace(go.Scatter(x=list(range(len(x))), y=x, mode='markers', marker={'size': 3}, name='Loss'))
    fig.add_trace(go.Scatter(x=list(range(len(x))), y=x.rolling(50).mean(), mode='lines', name='Rolling 50 mean'))
    break

fig.update_layout(
    title={
        "text": "One Experiment RL Hybrid Convergence Result",
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Number of Iteration",
    yaxis_title="L1 Loss Between Searched and Func Opt",
)

fig.show()

In [80]:
fig.write_image("/home/xyz/data/haoyan/bandits/bandits/image/mix/rl_hybrid.png")

In [81]:
import json

data_list = []
for n in range(5):
    try:
        with open(f'/home/xyz/data/haoyan/bandits/logs/bayesian_results_{func_name}_{n*128}.log.json', 'r') as f:
            data = json.loads('['+f.read().replace('\n', ',')[:-1]+']')
            l = []
            for x in data:
                tmp = {'reward': x['target']}
                tmp.update(x['params'])
                l.append(tmp)  
            df = pd.DataFrame(l)
            df['random_state'] = n * 128
            df['num_exp'] = df.index
            data_list.append(df)
    except FileNotFoundError:
        pass
    
df = pd.concat(data_list, ignore_index=True)
df['u'] = df['u'].apply(lambda u: int(int(u)))
df['x'] = df['x'].apply(lambda x: int(int(x)))

In [82]:
idx = df.groupby('random_state')['reward'].idxmax()
df.loc[idx]

,reward,u,x,y,random_state,num_exp
394,19.999811,2,0,-2.173211,0,394
845,19.999678,2,0,-3.911466,128,357
1060,20.000000,2,0,4.338745,256,101
1825,19.999913,2,0,0.503036,384,380
1955,19.999998,2,0,0.063471,512,14


In [83]:
df['reward-opt'] = (df['reward'] - opt).abs()
df['abs(reward-opt)<0.05'] = df['reward-opt'] < 0.05
df['abs(reward-opt)<0.1'] = df['reward-opt'] < 0.1
df['abs(reward-opt)<0.2'] = df['reward-opt'] < 0.2
tmp = df.groupby('random_state')[['abs(reward-opt)<0.05', 'abs(reward-opt)<0.1', 'abs(reward-opt)<0.2']].sum() / df.groupby('random_state').size().values.reshape((-1,1)) * 100
tmp.index.name = 'Bayesian Opt'
tmp

,abs(reward-opt)<0.05,abs(reward-opt)<0.1,abs(reward-opt)<0.2
Bayesian Opt,,,
0,5.327869,5.737705,7.172131
128,6.581741,8.917197,10.828025
256,3.086420,3.703704,5.555556
384,7.258065,9.677419,11.491935
512,4.329004,5.194805,6.493506


In [97]:
import plotly.graph_objs as go

fig = go.Figure()
moving_avg_list = []
for key, gp in df.groupby('random_state'):
    moving_avg_list.append(abs(gp.sort_values('num_exp')['reward'] - opt))

x = moving_avg_list[4]
y = moving_avg_list[0]
fig.add_trace(go.Scatter(x=list(range(len(y))), y=y, mode='markers', opacity=0.7, marker={'size': 5}, name='Loss'))
fig.add_trace(go.Scatter(x=list(range(len(y))), y=y.rolling(50).mean(), mode='lines', name='Rolling 50 mean'))

fig.update_layout(
    title={
        "text": "One Experiment Bayesian Opt Convergence Result",
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Number of Iteration",
    yaxis_title="L1 Loss Between Searched and Func Opt",
)
fig.update_yaxes(rangemode="tozero")

fig.show()

In [86]:
fig.write_image("/home/xyz/data/haoyan/bandits/bandits/image/mix/bo.png")

In [ ]:
tmp

In [91]:
from ast import literal_eval as make_tuple


l = []
last_random_state = -99999
names = ['random_state', 'num_exp', 'reward']
with open('/home/xyz/data/haoyan/bandits/logs/rl_results_mix.txt', 'r') as f:
    x = f.readline()
    while x:
        tmp = make_tuple(x)
        if last_random_state != tmp[0]:
            idx = 0
            last_random_state = tmp[0]
        tmp_dict = {
            'random_state': tmp[0],
            'nth_trial': tmp[1],
            'num_exp': idx,
            'u': tmp[2][0][0],
            'x': tmp[2][0][1],
            'y': tmp[2][0][2],
            'time': tmp[5],
        }
        tmp_dict['reward'] = tmp[4]
        l.append(tmp_dict)
        idx += 1
        x = f.readline()
df = pd.DataFrame(l)

In [92]:
df.groupby('random_state')['reward'].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

,count,mean,std,min,25%,50%,75%,90%,95%,99%,max
random_state,,,,,,,,,,,
0,1000.0,-3.251249,14.516619,-48.305269,-12.001836,0.364137,5.817808,14.005558,16.824195,18.984319,19.999369
128,1000.0,-3.019091,15.348554,-49.235437,-11.902952,0.534244,6.780169,15.817555,17.802454,19.137522,19.984218
256,1000.0,-3.338604,15.231186,-49.226147,-12.515189,0.654258,6.396591,15.060074,17.832731,19.039797,19.923613
384,1000.0,-3.103645,14.728062,-49.226147,-11.759944,0.494724,6.240256,15.060074,17.725665,19.607106,19.999369
512,1000.0,-3.341082,15.069187,-49.226147,-13.580834,0.494724,5.759667,15.229699,18.140299,19.135754,19.999369


In [93]:
df = pd.DataFrame(l)
idx = df.groupby('random_state')['reward'].idxmax()
df.loc[idx]

,random_state,nth_trial,num_exp,u,x,y,time,reward
252,0,252,252,2,0,0.025126,0.000148,19.999369
1753,128,753,753,2,0,-0.125628,0.000142,19.984218
2310,256,310,310,2,0,-0.276382,0.000157,19.923613
3038,384,38,38,2,0,0.025126,0.000149,19.999369
4692,512,692,692,2,0,0.025126,0.000143,19.999369


In [95]:
import plotly.graph_objs as go

fig = go.Figure()
moving_avg_list = []
for key, gp in df.groupby('random_state'):
    moving_avg_list.append(abs(gp.sort_values('num_exp')['reward'] - opt))

for x in moving_avg_list[0:1]:
    fig.add_trace(go.Scatter(x=list(range(len(x))), y=x, mode='markers', marker={'size': 3}, name='Loss'))
    fig.add_trace(go.Scatter(x=list(range(len(x))), y=x.rolling(50).mean(), mode='lines', name='Rolling 50 mean'))
    break

fig.update_layout(
    title={
        "text": "One Experiment RL Convergence Result",
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Number of Iteration",
    yaxis_title="L1 Loss Between Searched and Func Opt",
)
fig.update_yaxes(rangemode="tozero")
fig.show()

In [96]:
fig.write_image("/home/xyz/data/haoyan/bandits/bandits/image/mix/rl.png")

In [ ]:
idx = df.groupby('random_state')['bayesian_opt'].idxmax()
df.loc[idx]

In [ ]:
df.loc[idx]['num_exp'].mean()

In [ ]:
df.loc[idx]['num_exp'].mean()